In [1]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

import Data.Bits
import Data.Word
import Numeric
import Data.Char

newtype Binary a = Binary a deriving (Num, Eq, Ord, Bits, FiniteBits, Integral, Real, Enum)

instance (FiniteBits a, Integral a, Show a) => Show (Binary a) where
    show (Binary a) = let
        str = showIntAtBase 2 intToDigit a ""
        size = finiteBitSize (undefined `asTypeOf` a)
        in replicate (size - length str) '0' ++ str

In [2]:
type Bitmap = Binary Word16
type Hash = Binary Word32
type Shift = Int

bitsPerSubhash :: Int
bitsPerSubhash = 4

class Hashable a where
    hash :: a -> Hash

In [3]:
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE TypeSynonymInstances #-}

import Data.ByteArray.Hash
import Data.ByteString.Char8 (pack)

instance Hashable String where
    hash str = let
        FnvHash32 h = fnv1Hash (pack str)
        in Binary h

In [4]:
import Data.Vector (Vector)
import qualified Data.Vector as V

data HAMT key value
    = None
    | Leaf Hash key value
    | Many Bitmap (Vector (HAMT key value))
    deriving (Show)

empty = None

In [22]:
insertAt :: Vector a -> Int -> a -> Vector a
insertAt vector index a = V.take index vector V.++ V.singleton a V.++ V.drop index vector

updateAt :: Vector a -> Int -> a -> Vector a
updateAt vector index a = vector V.// [(index, a)]

deleteAt :: Vector a -> Int -> Vector a
deleteAt vector index = V.take index vector V.++ V.drop (index+1) vector

In [6]:
subHashMask :: Bitmap
subHashMask = bit (bitsPerSubhash) - 1

In [16]:
import Debug.Trace

insert_ :: Shift -> Hash -> key -> value -> HAMT key value -> HAMT key value

hashBitmap :: Hash -> Shift -> Bitmap
hashBitmap hash shift = let
    fragment = fromIntegral (hash `shiftR` shift) .&. subHashMask
    Binary position = fragment
    bitmap = Binary $ bit $ fromIntegral position
    in bitmap

hashIndex :: Hash -> Shift -> Bitmap -> Int
hashIndex hash shift bitmap = let
    fragment = fromIntegral (hash `shiftR` shift) .&. subHashMask
    mask = bit (fromIntegral fragment) - 1 :: Bitmap
    index = popCount (bitmap .&. mask)
    in index

insert_ shift hash key value None = Leaf hash key value

insert_ shift hash key value leaf@(Leaf leafHash leafKey leafValue)
    | hash == leafHash = Leaf hash key value
    | otherwise = insert_ shift hash key value $ Many (hashBitmap leafHash shift) (V.singleton leaf)

insert_ shift hash key value (Many bitmap vector)
    | bitmap .&. mask == 0 = let
        leaf = Leaf hash key value
        bitmap' = bitmap .|. mask
        vector' = insertAt vector index leaf
        in Many bitmap' vector'
    | otherwise = let
        subtree = vector V.! index
        subtree' = insert_ (shift + bitsPerSubhash) hash key value subtree
        in Many bitmap (updateAt vector index subtree')
    where 
        index = hashIndex hash shift bitmap
        mask  = hashBitmap hash shift

insert :: Hashable key => key -> value -> HAMT key value -> HAMT key value
insert key = insert_ 0 (hash key) key

In [8]:
leaf = insert_ 0 (hash "1") "1" 1 empty
leaf

Leaf 00000101000011000101110100101110 "1" 1

In [21]:
import Text.Show.Pretty
many = insert_ 0 (hash "10") "10" 2 leaf
many' = insert_ 0 (hash "100") "100" 3 many
pPrint many'

Many
  0100010000000000
  [ Leaf 00100000011101101010111101011010 "10" 2
  , Many
      0000001000000100
      [ Leaf 00000101000011000101110100101110 "1" 1
      , Leaf 01110100110101100000101010011110 "100" 3
      ]
  ]

In [20]:
import Prelude hiding (lookup)

lookup_ :: Shift -> Hash -> HAMT key value -> Maybe value
lookup_ shift hash None = Nothing

lookup_ shift hash (Leaf leafHash leafKey leafValue)
    | hash == leafHash = Just leafValue
    | otherwise = Nothing

lookup_ shift hash (Many bitmap vector)
    | bitmap .&. mask == 0 = Nothing
    | otherwise = let
        child = vector V.! index
        in lookup_ (shift+bitsPerSubhash) hash child
    where
        index = hashIndex hash shift bitmap
        mask  = hashBitmap hash shift


lookup :: Hashable key => key -> HAMT key value -> Maybe value
lookup key = lookup_ 0 (hash key)

lookup "1" many'

Just 1

In [25]:
delete_ :: Shift -> Hash -> HAMT key value -> HAMT key value

delete_ shift hash None = None

delete_ shift hash leaf@(Leaf leafHash leafKey leafValue)
    | hash == leafHash = None
    | otherwise = leaf

delete_ shift hash many@(Many bitmap vector)
    | bitmap .&. mask == 0 = many
    | otherwise = let
        subtree = vector V.! index
        subtree' = delete_ (shift+bitsPerSubhash) hash subtree
        in case subtree' of
            None -> if V.length vector == 1
                then None
                else Many (bitmap .&. (complement mask)) (deleteAt vector index)
            Leaf{} -> if V.length vector == 1
                then subtree'
                else  Many bitmap (updateAt vector index subtree')
            Many{} -> Many bitmap (updateAt vector index subtree')
    where
        index = hashIndex hash shift bitmap
        mask  = hashBitmap hash shift

delete :: Hashable key => key -> HAMT key value -> HAMT key value
delete key = delete_ 0 (hash key)

In [28]:
pPrint $ delete "100" $ delete "1" many'

Many
  0000010000000000 [ Leaf 00100000011101101010111101011010 "10" 2 ]